# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-04 20:22:44] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-04 20:22:44] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-04 20:22:44] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-04 20:22:47] INFO server_args.py:1615: Attention backend not specified. Use fa3 backend by default.


[2026-01-04 20:22:47] INFO server_args.py:2502: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=80 avail_mem=51.63 GB):  20%|██        | 4/20 [00:00<00:01, 14.43it/s]

Capturing batches (bs=64 avail_mem=51.62 GB):  35%|███▌      | 7/20 [00:00<00:00, 18.51it/s]

Capturing batches (bs=24 avail_mem=56.26 GB):  65%|██████▌   | 13/20 [00:00<00:00, 15.67it/s]

Capturing batches (bs=4 avail_mem=56.20 GB):  75%|███████▌  | 15/20 [00:01<00:00, 15.51it/s]

Capturing batches (bs=1 avail_mem=56.19 GB): 100%|██████████| 20/20 [00:01<00:00, 16.65it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Aileen and I am a full-time student, currently enrolled at King's College London. I am a first year computer science undergraduate and I'm quite a curious student with a lot of passion for technology, particularly the fields of AI and machine learning. I have a great interest in the data science and machine learning community, and I am always keen to learn and expand my knowledge. I have a lot of experience in both data analysis and programming and I am eager to apply my skills in the field of AI and machine learning to real-world problems. What are some ways that I can improve my skills in these areas and stay up to date with
Prompt: The president of the United States is
Generated text:  seeking the support of the Democratic Party. The Democratic Party is composed of the following groups: the Senate, the House of Representatives, the White House, the United States Postal Service, and the National Guard. The president is willing to meet with r

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Occupation]. I'm a [Skill/Ability] who has been [Number of Years] years in this field. I'm passionate about [Why I'm passionate about this field]. I'm always looking for new challenges and opportunities to grow and learn. I'm a [Favorite Activity] and I enjoy [What I enjoy doing]. I'm [What I'm looking forward to doing next]. I'm [What I hope to achieve in the future]. Thank you for asking! [Name] looks forward to having the opportunity to meet you and learn more about you. [

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. It is the largest city in France and the second-largest city in the European Union. Paris is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and the Arc de Triomphe. The city is also famous for its cuisine, fashion, and art. Paris is a cultural and historical center that attracts millions of visitors each year. It is a major economic and financial center as well. The city is home to many world-renowned museums, theaters, and art galleries. Paris is a popular tourist destination and a major hub for international business and diplomacy. It is a city

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction and impact on society. Here are some of the most likely trends:

1. Increased use of AI in healthcare: AI is already being used in healthcare to improve patient outcomes, reduce costs, and increase efficiency. As AI technology continues to improve, we can expect to see even more widespread adoption in healthcare, with more sophisticated algorithms and machine learning techniques being used to diagnose and treat diseases.

2. AI in finance: AI is already being used in finance to improve risk management, fraud detection, and portfolio optimization. As AI technology continues to evolve, we



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I am a [current role]. I'm [age] years old, and I have a [background, such as military, academic, etc.]. I love [why] and have a passion for [what], and I enjoy [why] with friends. My best attribute is [what]. I'm always looking for [why] and would love to work on [something]. I'm an [career path] with [experience level] of experience in [one or two career paths]. What brings you to this job?

[Name] was first introduced to this company by [who] when [when], where

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, also known as “La Palais Royal” or “le Palais-Royal” and is located in the western part of the country, just north of the Seine river. The city is known for its rich history, beautiful architecture, and bustling city life. It is the largest city in Europe and is home to many renowned

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

]

 and

...

 in

 the

 future

.

 This

In [6]:
llm.shutdown()